In [22]:
import pandas as pd
import numpy as np

In [5]:
test = pd.read_pickle('data_test.p')

object_cols = list(test.dtypes[test.dtypes == type(object())].index)
drop_cols = object_cols + ['date', 'average_stars']

In [6]:
train = pd.read_pickle('data_train.p').drop(drop_cols, axis=1)

In [7]:
train_X = train.drop('stars', axis=1)
mean = train_X.mean()
std = train_X.std()

def transform(data):
    return ((data - mean)/std).fillna(0)

In [8]:
train_X = transform(train_X)
test = pd.read_pickle('data_test.p').drop(drop_cols, axis=1)
val = pd.read_pickle('data_val.p').drop(drop_cols, axis=1)

In [9]:
from sklearn.neural_network import MLPRegressor
train_y = train['stars']
val_X = transform(val.drop('stars', axis=1))
val_y = val['stars']

def get_preds(clf):
    preds = clf.predict(val_X)
    preds[preds > 5] = 5
    preds[preds < 1] = 1
    return preds

def get_accuracy(solver, alpha, hidden_layer_sizes, activation='relu', batch_size=200, 
                 learning_rate='constant'):
    clf = MLPRegressor(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes)
    clf.fit(train_X, train_y)
    return np.mean((get_preds(clf) - val_y) ** 2)

In [10]:
n = train_X.shape[1]
hidden_layers = []

for i in range(2, 10):
    hidden_layers.append(tuple(np.linspace(101, 1, i).round().astype(int)[1:-1]))

hidden_layer_accuracies = []
# these accuracies perform comparably to the smaller sized layers
for sizes in hidden_layers:
    accuracy = get_accuracy(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=sizes)
    print(accuracy)
    hidden_layer_accuracies.append(accuracy)

1.4900943810264935
1.4802598046420592
1.4885726264549823
1.4970355466418703
1.4884239489758369
1.485299627797674
1.504058780175919
1.4849604291674259


In [11]:
hidden_layers = [(5,2), (5), (5, 3, 2)]

accuracies = []
for sizes in hidden_layers:
    accuracy = get_accuracy(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=sizes)
    print(accuracy)
    accuracies.append(accuracy)

1.4773759104699247
1.4787658632249203
1.4802370520965316


In [12]:
# accuracy for 0 hidden layer (i.e. logistic regression)
accuracy = get_accuracy(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=())
print(accuracy)

1.4901187545606067


In [13]:
# results of baseline, using either average stars of the user or restaurant
print(np.mean((test.avg_stars_x - test['stars']) ** 2))
print(np.mean((test.avg_stars_y - test['stars']) ** 2))

1.548369652113151
1.7454044534779938


In [14]:
activations = np.array(['logistic', 'tanh', 'relu'])
solvers = np.array(['adam', 'sgd', 'lbfgs'])
alphas = np.array([1e-5, 1e-4, 1e-3])
batch_sizes = np.array([100, 200, 300])
learning_rates = np.array(['constant', 'invscaling', 'adaptive'])

accuracies = pd.DataFrame(\
    columns=['activation', 'solver', 'alpha', 'batch_size', 'learning_rate', 'accuracy'])

for i in range(10):
    solver = np.random.choice(solvers)
    alpha = np.random.choice(alphas)
    activation = np.random.choice(activations)
    batch_size = np.random.choice(batch_sizes)
    learning_rate = np.random.choice(learning_rates)
    
    accuracy = get_accuracy(solver, alpha, (5), activation, batch_size, learning_rate)
    
    accuracies = accuracies.append({'activation': activation, 'solver': solver,\
        'alpha': alpha, 'batch_size': batch_size, 'learning_rate': learning_rate,\
        'accuracy': accuracy}, ignore_index=True)
    print(accuracies.iloc[-1])
    print('\n')

activation           tanh
solver                sgd
alpha               1e-05
batch_size            200
learning_rate    adaptive
accuracy          1.49052
Name: 0, dtype: object


activation             relu
solver                  sgd
alpha                0.0001
batch_size              100
learning_rate    invscaling
accuracy            1.48708
Name: 1, dtype: object


activation           relu
solver               adam
alpha               1e-05
batch_size            200
learning_rate    constant
accuracy          1.47564
Name: 2, dtype: object


activation           relu
solver                sgd
alpha               1e-05
batch_size            100
learning_rate    constant
accuracy          1.48503
Name: 3, dtype: object


activation           relu
solver              lbfgs
alpha               1e-05
batch_size            300
learning_rate    adaptive
accuracy          1.47742
Name: 4, dtype: object


activation           tanh
solver                sgd
alpha              0.0001
batch

In [15]:
accuracies

,activation,solver,alpha,batch_size,learning_rate,accuracy
0,tanh,sgd,0.00001,200,adaptive,1.490520
1,relu,sgd,0.00010,100,invscaling,1.487084
2,relu,adam,0.00001,200,constant,1.475644
3,relu,sgd,0.00001,100,constant,1.485032
4,relu,lbfgs,0.00001,300,adaptive,1.477415
5,tanh,sgd,0.00010,200,adaptive,1.491800
6,logistic,sgd,0.00001,200,invscaling,1.490088
7,logistic,lbfgs,0.00100,300,constant,1.475184
8,relu,sgd,0.00010,200,adaptive,1.483548
9,logistic,lbfgs,0.00010,200,invscaling,1.485110


In [16]:
hidden_layers = [[5], [5, 2]]

for i in range(2, 10):
    hidden_layers.append(np.linspace(101, 1, i).round().astype(int)[1:-1])

hidden_layers = np.array(hidden_layers)

accuracies_with_hidden_layers = pd.DataFrame(columns=['activation', 'solver', 'alpha', \
    'batch_size', 'learning_rate', 'hidden_layers', 'accuracy'])

for i in range(100):
    solver = np.random.choice(solvers)
    alpha = np.random.choice(alphas)
    activation = np.random.choice(activations)
    batch_size = np.random.choice(batch_sizes)
    learning_rate = np.random.choice(learning_rates)
    hidden_layer_sizes = tuple(np.random.choice(hidden_layers))
    
    accuracy = get_accuracy(\
        solver, alpha, hidden_layer_sizes, activation, batch_size, learning_rate)
    
    accuracies_with_hidden_layers = accuracies_with_hidden_layers.append(\
        {'activation': activation, 'solver': solver, 'alpha': alpha, 'batch_size': batch_size, \
         'learning_rate': learning_rate, 'hidden_layers': hidden_layer_sizes, \
         'accuracy': accuracy}, ignore_index=True)
    print(accuracies_with_hidden_layers.iloc[-1])
    print('\n')

activation                           logistic
solver                                  lbfgs
alpha                                  0.0001
batch_size                                100
learning_rate                        constant
hidden_layers    (88, 76, 64, 51, 38, 26, 14)
accuracy                              1.48054
Name: 0, dtype: object


activation           relu
solver              lbfgs
alpha              0.0001
batch_size            200
learning_rate    constant
hidden_layers        (5,)
accuracy          1.48155
Name: 1, dtype: object


activation                           relu
solver                              lbfgs
alpha                               1e-05
batch_size                            300
learning_rate                    adaptive
hidden_layers    (87, 72, 58, 44, 30, 15)
accuracy                          1.48314
Name: 2, dtype: object


activation           relu
solver               adam
alpha               0.001
batch_size            300
learning_rate    consta

/Users/joshuameisel/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


activation                   relu
solver                        sgd
alpha                       0.001
batch_size                    300
learning_rate            adaptive
hidden_layers    (81, 61, 41, 21)
accuracy                   1.5762
Name: 19, dtype: object


activation       logistic
solver               adam
alpha              0.0001
batch_size            100
learning_rate    adaptive
hidden_layers        (5,)
accuracy           1.4838
Name: 20, dtype: object


activation       logistic
solver              lbfgs
alpha               0.001
batch_size            300
learning_rate    adaptive
hidden_layers       (51,)
accuracy          1.47877
Name: 21, dtype: object


activation       logistic
solver                sgd
alpha               0.001
batch_size            100
learning_rate    constant
hidden_layers    (68, 34)
accuracy          1.48755
Name: 22, dtype: object


activation         logistic
solver                 adam
alpha                 0.001
batch_size              100


activation       logistic
solver                sgd
alpha               1e-05
batch_size            300
learning_rate    adaptive
hidden_layers          ()
accuracy          4.53287
Name: 52, dtype: object


activation                           relu
solver                              lbfgs
alpha                               0.001
batch_size                            100
learning_rate                  invscaling
hidden_layers    (87, 72, 58, 44, 30, 15)
accuracy                          1.50038
Name: 53, dtype: object


activation                           tanh
solver                              lbfgs
alpha                              0.0001
batch_size                            200
learning_rate                    constant
hidden_layers    (87, 72, 58, 44, 30, 15)
accuracy                          1.49641
Name: 54, dtype: object


activation       logistic
solver               adam
alpha              0.0001
batch_size            100
learning_rate    constant
hidden_layers      (5,

/Users/joshuameisel/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


activation                   logistic
solver                           adam
alpha                          0.0001
batch_size                        100
learning_rate              invscaling
hidden_layers    (84, 68, 51, 34, 18)
accuracy                      1.60712
Name: 60, dtype: object




KeyboardInterrupt: 

In [20]:
accuracies_with_hidden_layers.to_csv('nn_accuracies.csv')

In [23]:
accuracies_with_hidden_layers = pd.read_csv('nn_accuracies.csv')

In [25]:
accuracies_with_hidden_layers.sort_values('accuracy')

,Unnamed: 0,activation,solver,alpha,batch_size,learning_rate,hidden_layers,accuracy
34,34,relu,lbfgs,0.00001,200,invscaling,"(81, 61, 41, 21)",1.478256
55,55,logistic,adam,0.00010,100,constant,"(5, 2)",1.478758
21,21,logistic,lbfgs,0.00100,300,adaptive,"(51,)",1.478774
45,45,relu,lbfgs,0.00010,200,adaptive,"(51,)",1.478978
57,57,logistic,lbfgs,0.00100,100,constant,"(87, 72, 58, 44, 30, 15)",1.479692
59,59,logistic,sgd,0.00100,100,constant,"(51,)",1.480007
38,38,tanh,lbfgs,0.00100,200,adaptive,"(5, 2)",1.480179
39,39,tanh,lbfgs,0.00100,200,adaptive,"(51,)",1.480361
0,0,logistic,lbfgs,0.00010,100,constant,"(88, 76, 64, 51, 38, 26, 14)",1.480544
36,36,relu,lbfgs,0.00010,200,invscaling,"(87, 72, 58, 44, 30, 15)",1.480805
